In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./b.part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[['JOIN_MPN', 'Part_Type', 'Part_Key']]
input_.drop_duplicates(['JOIN_MPN'],
                       keep='first',
                       ignore_index=True)

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

import numpy as np

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                jsn = '{"partData":{"listing_data_essential":{"parttype":"' + input_.loc[i, 'Part_Type'] + '","partkey":"' + input_.loc[i, 'Part_Key'] + '"}}}'

                payload = {'func': 'getbuyersguide',
                           'payload': jsn,
                           'api_json_request': '1'}
                
                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=payload,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['buyersguidepieces']['body']

                soup = BeautifulSoup(html, 'lxml')

                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_vehicle = []
                list_tr = html.xpath('//div[contains(@class, "buyersguide-nested")]/div/table[contains(@class, "nobmp")]/tr')
                for tr in list_tr:
                    list_td = [text.strip() for text in tr.xpath('./td/text()')]
                    list_vehicle.append(' '.join(list_td).strip())

                vehicle = '\n'.join(list_vehicle)

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status == 'ok':
            df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                                     'Vehicle': vehicle}])

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
        else:
            df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                                     'Part_Type': input_.loc[i, 'Part_Type'],
                                     'Part_Key': input_.loc[i, 'Part_Key']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Part_Type'] + ' - ' + input_.loc[i, 'Part_Key'] + '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_MPN'], ascending=[True])
output_correct.to_excel('./c.vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['JOIN_MPN'], ascending=[True])
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：2

256 - 15138777  <->  [ok] - 剩余数量：0 - 当前时间：13:50:53
260 - 368851  <->  [ok] - 剩余数量：0 - 当前时间：13:50:53

爬虫输出中

爬虫结束
